In [1]:
import pandas as pd
import os
import numpy as np
import geojson
import cv2
from json.decoder import JSONDecodeError


In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [6]:
PATH_hovernet = '/media/user/Seagate_RNA/PCNSL_PSL/3_hovernet_normalisation_reinhard_224/'
PATH_output = '/media/user/Seagate_RNA/PCNSL_PSL/4_df_count_cells/'

In [7]:
patients = os.listdir(PATH_hovernet)
print(len(patients))

106


In [8]:
columns_df = {'patient' : [], 'patch' : [], 'cells_tot' : [], 'cells_filtre_bords' : [], 'cells_filtre_diametre' : [], 'cells_filtre_tot' : []}
df = pd.DataFrame.from_dict(columns_df)
df

,patient,patch,cells_tot,cells_filtre_bords,cells_filtre_diametre,cells_filtre_tot


In [9]:
#removing nuclei located at the border and nuclei which don't have a number of pixels  in the range [32:350] 
width = 224
marge = 1
res = 0.23

nbre_pix_inf = 32
nbre_pix_sup = 350

In [10]:
from PIL import Image, ImageDraw
def function_cell(cell) :
    width = 224
    poly_cell = []
    for x in cell["contour"] :
        poly_cell.append((x[0], x[1]))
    width = width
    height = width
    
    mask_cell = np.zeros((width, width), dtype = int)
    
    img = Image.new('L', (width, height), 0)
    ImageDraw.Draw(img).polygon(poly_cell, outline = 1, fill=1)
    mask = np.array(img)
    mask_cell += mask
    
    mask_cell = np.array(mask_cell, dtype = 'uint8')

    
    return mask_cell

In [11]:
def nbre_pixels(cnt1):
    mask_cell = function_cell(cnt1)
    number_of_white_pix = np.sum(mask_cell != 0)
    return number_of_white_pix

In [15]:
for patient in patients :
    if patient not in [''] :
        if not os.path.exists(PATH_output+patient+'_.csv'):# and patient not in ['19B2703-V']:
           # print(patient)
            PATH_json = PATH_hovernet+patient+'/json/' 
            columns_df = {'patient' : [], 'patch' : [], 'cells_tot' : [], 'cells_filtre_bords' : [], 'cells_filtre_diametre' : [], 'cells_filtre_tot' : []}
            df = pd.DataFrame.from_dict(columns_df)
            jsons = os.listdir(PATH_json)
            #print(len(jsons))
            for patch in jsons : 
                if patch not in ['']:#liste:
                    #print(patch)
                    with open(PATH_json+patch) as f :
                        nuclei = geojson.load(f)


                    key_keep = []
                    key_not_keep_bords = []
                    key_not_keep_diametre = []
                    key_not_keep = []

                    cells_tot = len(list(nuclei['nuc'].keys()))
                    
                    for key in list(nuclei['nuc'].keys()) :
                        for x in nuclei['nuc'][key]['contour'] :
                            if x[0] < marge or x[0] > (width-marge) or x[1] < marge or x[1] > (width-marge) :
                                key_not_keep_bords.append(key)     
                            pix_count = nbre_pixels(nuclei['nuc'][key])

                            if pix_count < nbre_pix_inf or pix_count > nbre_pix_sup : #
                                key_not_keep_diametre.append(key)

                            key_not_keep_bords = list(set(key_not_keep_bords))
                            key_not_keep_diametre = list(set(key_not_keep_diametre))
                            key_not_keep = key_not_keep_bords + key_not_keep_diametre
                            key_not_keep = list(set(key_not_keep))

                        for x in nuclei['nuc'][key]['contour'] :
                            if key not in key_not_keep  :
                                key_keep.append(key) 

                    new_row = {'patient' : patient, 'patch' : patch, 'cells_tot' : cells_tot, 'cells_filtre_bords' : cells_tot - len(key_not_keep_bords), 'cells_filtre_diametre' : cells_tot - len(key_not_keep_diametre), 'cells_filtre_tot' : cells_tot - len(key_not_keep)}
                    df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

            df = df.sort_values(by=['cells_filtre_tot'], ascending = False)
            df.to_csv(PATH_output+patient+'.csv')
        else:
            print(patient, 'déjà traité')

# To have some statistics

In [ ]:
print(df_tot['cells_tot_mean'].mean())

In [ ]:
df_tot['cells_filtre_bords_mean'].mean()
print(df_tot['cells_filtre_bords_mean'].mean()*100/df_tot['cells_tot_mean'].mean())

In [ ]:
df_tot['cells_filtre_diametre_mean'].mean()
print(df_tot['cells_filtre_diametre_mean'].mean()*100/df_tot['cells_tot_mean'].mean())

In [ ]:
df_tot['cells_filtre_tot_mean'].mean()
print(df_tot['cells_filtre_tot_mean'].mean()*100/df_tot['cells_tot_mean'].mean())